#### Variables coded in this notebook: 18, 1373, 8345 et seq, 8780 et seq

In [4]:
import os, pandas as pd, re

In [5]:
exportdate = 20180327
projectname = 'repract'

In [6]:
df = pd.read_csv(f'../../data/{exportdate}{projectname}.csv')
df.head(2)

,lfdn,external_lfdn,tester,dispcode,lastpage,quality,duration,v_7039,v_7040,v_7041,...,output_mode,javascript,flash,session_id,language,cleaned,ats,datetime,date_of_last_access,date_of_first_mail
0,106,0,no tester,Completed after break (32),2138658,NotShown,-1,NotShown,NotShown,0,...,HTML,NotShown,NotShown,3bb21c1b318e2f6b87557566bdd6b4d9,English,Not cleaned,1515411510,2018-01-08 11:38:30,2018-01-08 13:07:14,0000-00-00 00:00:00
1,131,0,no tester,Completed (31),2138658,NotShown,3805,NotShown,NotShown,NotShown,...,HTML,NotShown,NotShown,fc38f6556787a459c2cc604abf799448,English,Not cleaned,1515667019,2018-01-11 10:36:59,2018-01-11 11:40:24,0000-00-00 00:00:00


In [7]:
basedir = '../../data/freetext'
freetextfiles = os.listdir(basedir)
dfs = {file[:-4]:pd.read_csv(f'{basedir}/{file}') for file in freetextfiles}

In [8]:
dfs.keys()

dict_keys(['.DS_S', 'v_11', 'v_1373', 'v_16', 'v_18', 'v_19', 'v_6', 'v_8345etseq', 'v_8780etseq'])

Leaving this here temporarily merely for reference!

In [120]:
def add_unique_code(df, func, varname):
    df[f'{varname}_coded'] = func(df[varname])
    return df

In [79]:
codedir = '../../analysis/freetext'
def write_coded(df, varname):
    filepath = f'{codedir}/{varname}_coded.csv'
    df.to_csv(f'{codedir}/{varname}_coded.csv', index=False)
    print(f'File stored at {filepath}.')

For long entries, we actually want to enable multiple tagging, therefore, we want to produce a long-form DF

In [24]:
def code_var_long(df, index_col, coding_col, codes, match_func):
    new_df = pd.DataFrame(columns=[index_col, f'{coding_col}_code'])
    for row in df[[index_col, coding_col]].iterrows():
        for code in codes:
            if match_func(row[1][1], code):
                new_df = new_df.append(pd.DataFrame({index_col:[row[1][0]],f'{coding_col}_code':[code]}))
    return new_df.reset_index().drop('index', axis=1)

In [34]:
# Test bogus
def match_v18(statement, code):
    if len(str(statement)) < len(code):
        return True
    else:
        return False
    
def v_18_codes():
    return ['Code_A', 'Code_B']

In [35]:
code_var_long(dfs['v_18'], 'lfdn', 'v_18', v_18_codes(), match_v18)

,lfdn,v_18_code
0,224,Code_A
1,224,Code_B
2,409,Code_A
3,409,Code_B
4,411,Code_A
5,411,Code_B
6,1027,Code_A
7,1027,Code_B
